## Synthetic Image Generation
Based on Method 1 - COCO Annotations

In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image, ImageOps
import skimage.io as io
import skimage.transform as transform
from skimage import measure,draw,data
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pycocotools.coco import COCO
import pycocotools.mask as maskutil
from itertools import groupby
from imantics import Polygons, Mask
from shapely.geometry import Polygon, MultiPolygon
import json
import random

%matplotlib inline
matplotlib.use('Agg')

In [2]:
def foregroundAug(foreground):
    
    angle = np.random.randint(-10,10)*(np.pi/180.0) # Convert to radians
    zoom = np.random.random()*0.4 + 0.8             # Scale in range [0.8,1.2)
    
    t_x = np.random.randint(0, int(foreground.shape[1]/3))
    t_y = np.random.randint(0, int(foreground.shape[0]/3))

    tform = transform.AffineTransform(scale=(zoom,zoom),
                                rotation=angle,
                                translation=(t_x, t_y))
    foreground = transform.warp(foreground, tform.inverse)

    # Random horizontal flip with 0.5 probability
    if(np.random.randint(0,100)>=50):
        foreground = foreground[:, ::-1]
        
    return foreground

In [3]:
def getForegroundMask(foreground):
    mask_new = foreground.copy()[:,:,0]
    mask_new[mask_new>0] = 1
    return mask_new

In [4]:
def compose(foreground, mask, background):
    # Resize background to match the size of foreground
    background = transform.resize(background, foreground.shape[:2])

    # Subtract the foreground area from the background
    background = background*(1 - mask.reshape(foreground.shape[0], foreground.shape[1], 1))

    # Paste the foreground on the background
    composed_image = background + foreground 
  
    return composed_image

In [5]:
def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour

In [6]:
def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation
    Args:
        binary_mask: a 2D binary numpy array where '1's represent the object
        tolerance: Maximum distance from original points of polygon to approximated
            polygonal chain. If tolerance is 0, the original coordinate array is returned.
    """
    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)
    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        poly = Polygon(contour)
        #poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        #segmentation = contour.ravel().tolist()
        # after padding and subtracting 1 we may get -0.5 points in our segmentation
        #segmentation = [0 if i < 0 else i for i in segmentation]
        #polygons.append(segmentation)

    return polygons

In [7]:
annFile = 'instances_val2017.json'
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.85s)
creating index...
index created!


In [42]:
#category = 'cat'        # 1
#category = 'dog'        # 2
#category = 'horse'      # 3
#category = 'sheep'      # 4
#category = 'cow'        # 5
#category = 'elephant'   # 6
#category = 'bird'       # 7
#category = 'bear'       # 8 
#category = 'zebra'      # 9
category = 'giraffe'    # 10

In [43]:
# get all images containing given categories
catIds = coco.getCatIds(catNms = [category]);
imgIds = coco.getImgIds(catIds = catIds);
print("Number of images containing all filter classes:", len(imgIds))

Number of images containing all filter classes: 101


In [44]:
# Iteratively read foreground images from COCO
count = 5

index_list = list(range(0,len(imgIds)))

while(count):   
    random_index = random.choice(index_list)

    img = coco.loadImgs(imgIds[random_index])[0]
    I = io.imread(img['coco_url'])/255.0
    
    # Load annotations via COCO API
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    # Select proper annotated instances with size and segmentation
    if (len(anns[0]['segmentation']) > 1 or 
        len(anns) > 1 or
        anns[0]['area'] < 15000 or 
        anns[0]['area'] > 80000):
        continue
        
    print(random_index, img['file_name'][6:])  
    
    mask = np.zeros((img['height'],img['width']))
    for n in range(len(anns)):
        mask = np.maximum(coco.annToMask(anns[n]), mask)

    mask = mask.reshape(I.shape[0], I.shape[1], 1)

    foreground = mask*I    

    
    DIR = "background/"
    imagePaths = [os.path.join(DIR, f) for f in os.listdir(DIR)]
    if DIR + '.DS_Store' in imagePaths:
        imagePaths.remove(DIR + '.DS_Store')
    
    for j in range(len(imagePaths)):   # Iteratively read background images
        
        background = io.imread(imagePaths[j],plugin='matplotlib')/255.0
        background = np.array(background)
        
        if background.shape[2] != foreground.shape[2]:   # Should be with same dimension
            continue
        
        foreground_new = foregroundAug(foreground)
        

        mask_new = getForegroundMask(foreground_new)        
        #plt.imsave("data1/output/mask/{}_{}.png".format(i, j), mask_new)
        
        polygon = binary_mask_to_polygon(mask_new)
        if len(polygon) == 0:
            continue
        

        composed_image = compose(foreground_new, mask_new, background)
    
    
        composed_image = tf.clip_by_value(composed_image, 0.0, 1.0)
        
        plt.imsave("syn_img/{}/{}_{}_{}.jpg".format(category, category, img['file_name'][6:12], j), 
                   composed_image.numpy())    
        
        

        my_anno = list(polygon[0].exterior.coords)
        
        
        points_list = []
        for seg in range(len(my_anno)):
            points_list.append(list(my_anno[seg]))
            
        data = {}
        data['shapes'] = []
        data['shapes'].append({
            'label': category,
            'points': points_list,
            'shape_type': 'polygon'
        })
        data['imagePath'] = "{}_{}_{}.jpg".format(category, img['file_name'][6:12], j)
        data['imageHeight'] = img['height']
        data['imageWidth'] = img['width']

        with open('syn_img/{}/{}_{}_{}.json'.format(category, category, 
                                                   img['file_name'][6:12], j), 'w') as outfile:
            json.dump(data, outfile, indent=2)
            
    index_list.remove(random_index)
    count = count - 1

    

55 442661.jpg
91 537053.jpg
76 223130.jpg
12 433243.jpg
60 259382.jpg
